In [1]:
import qiskit
import qiskit_aer
import qiskit_nature
import qiskit_nature_pyscf
import qiskit_algorithms
import qiskit_nature_pyscf
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.circuit.library import EfficientSU2
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_aer import  AerSimulator
# from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from scipy.linalg import eigh
from qiskit_nature.second_q.circuit.library import HartreeFock
from scipy.linalg import expm
from qiskit.quantum_info import Statevector
from numpy import pi
import pylatexenc
import numpy as np
import matplotlib.pyplot as plt

In [2]:
print(f"qiskit version: {qiskit.__version__}")
print(f"qiskit-aer version: {qiskit_aer.__version__}")
print(f"qiskit-algorithms version: {qiskit_algorithms.__version__}")
print(f"qiskit-nature version: {qiskit_nature.__version__}")
print(f"qiskit-nature-pyscf version: {qiskit_nature_pyscf.__version__}")

qiskit version: 1.4.0
qiskit-aer version: 0.17.0
qiskit-algorithms version: 0.3.1
qiskit-nature version: 0.7.2
qiskit-nature-pyscf version: 0.4.0


In [3]:
# Define the Li molecule
molecule = MoleculeInfo(
    symbols=["Li", "H"],
    coords=([0.0, 0.0, 0.0], [0.0, 0.0, 1.59]),  # Approximate bond distance for He2
    multiplicity=1,  # Singlet state
    charge=0
)

# Set up the PySCF driver
mdriver = PySCFDriver.from_molecule(
    molecule=molecule,
    basis="sto3g"
)


# You can now use the driver to run a calculation
electronic_structure_problem = mdriver.run()

In [4]:
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(electronic_structure_problem.second_q_ops()[0])

In [5]:
from KQSCI_module import *

In [7]:
evolved_state_file = "state_probabilities.txt"
sample_space_size = 3
kqsci_size = 3
num_qubits = 12
state_probabilities = read_states(evolved_state_file)
eigenvalues, eigenvectors = kqsci_eign_SVD(state_probabilities, num_qubits, sample_space_size, kqsci_size, qubit_hamiltonian)

In [8]:
0.998447567774 + eigenvalues

array([-7.88026774, -7.12352517, -6.79921234])

In [9]:
evolved_state_file = "state_probabilities.txt"
state_probabilities = read_states(evolved_state_file)
max_d = 20
max_k = 10


In [10]:
with open("LiH_KQSCI_result_d_k_SVD.txt", "w") as file:
    for j in range(3, max_k):
        for i in range(1, max_d):
            
            sample_space_size = i
            kqsci_size = j
            num_qubits = 12
            
            eigenvalues, eigenvectors = kqsci_eign(state_probabilities, num_qubits, sample_space_size, kqsci_size, qubit_hamiltonian)
            gs_energy = eigenvalues[0] + 0.998447567774
            error = (7.882472287557 + eigenvalues[0] + 0.998447567774) / 7.882472287557 * 100
            file.write(f"sample_space_size(d): {sample_space_size}, kqsci_size(k): {kqsci_size}, gs_energy: {gs_energy}, error: {error}\n")
            print(f"sample_space_size(d): {sample_space_size}, kqsci_size(k): {kqsci_size}, gs_energy: {gs_energy}, error: {error}")
            file.flush()  # Ensure the data is written to the file immediately

sample_space_size(d): 1, kqsci_size(k): 3, gs_energy: -7.862174819467984, error: 0.2575012933576394
sample_space_size(d): 2, kqsci_size(k): 3, gs_energy: -7.876631327282255, error: 0.07410061287453647
sample_space_size(d): 3, kqsci_size(k): 3, gs_energy: -7.880266020192711, error: 0.027989535310792426
sample_space_size(d): 4, kqsci_size(k): 3, gs_energy: -7.880264641653427, error: 0.028007023977218545
sample_space_size(d): 5, kqsci_size(k): 3, gs_energy: -7.880409015200782, error: 0.026175447003794646
sample_space_size(d): 6, kqsci_size(k): 3, gs_energy: -7.871603512692416, error: 0.13788535459543014
sample_space_size(d): 7, kqsci_size(k): 3, gs_energy: -7.871917816639184, error: 0.13389797683751745
sample_space_size(d): 8, kqsci_size(k): 3, gs_energy: -7.871718823055375, error: 0.13642248408028168
sample_space_size(d): 9, kqsci_size(k): 3, gs_energy: -7.8714862817129605, error: 0.1393725907718331
sample_space_size(d): 10, kqsci_size(k): 3, gs_energy: -7.871503795066174, error: 0.13915

In [11]:
with open("LiH_KQSCI_result_d_k_QR.txt", "w") as file:
    for j in range(3, max_k):
        for i in range(1, max_d):
            
            sample_space_size = i
            kqsci_size = j
            num_qubits = 12
            
            eigenvalues, eigenvectors = kqsci_eign_QR(state_probabilities, num_qubits, sample_space_size, kqsci_size, qubit_hamiltonian)
            gs_energy = eigenvalues[0] + 0.998447567774
            error = (7.882472287557 + eigenvalues[0] + 0.998447567774) / 7.882472287557 * 100
            file.write(f"sample_space_size(d): {sample_space_size}, kqsci_size(k): {kqsci_size}, gs_energy: {gs_energy}, error: {error}\n")
            print(f"sample_space_size(d): {sample_space_size}, kqsci_size(k): {kqsci_size}, gs_energy: {gs_energy}, error: {error}")
            file.flush()  # Ensure the data is written to the file immediately

sample_space_size(d): 1, kqsci_size(k): 3, gs_energy: -7.8621748197633385, error: 0.25750128961065694
sample_space_size(d): 2, kqsci_size(k): 3, gs_energy: -7.876631344028479, error: 0.0741004004256581
sample_space_size(d): 3, kqsci_size(k): 3, gs_energy: -7.880267744959103, error: 0.0279676542774185
sample_space_size(d): 4, kqsci_size(k): 3, gs_energy: -7.880267744959103, error: 0.0279676542774185
sample_space_size(d): 5, kqsci_size(k): 3, gs_energy: -7.88041165958115, error: 0.02614189940259108
sample_space_size(d): 6, kqsci_size(k): 3, gs_energy: -7.871603527422676, error: 0.13788516772183085
sample_space_size(d): 7, kqsci_size(k): 3, gs_energy: -7.871917828615916, error: 0.1338978248961968
sample_space_size(d): 8, kqsci_size(k): 3, gs_energy: -7.871718833902098, error: 0.13642234647468998
sample_space_size(d): 9, kqsci_size(k): 3, gs_energy: -7.8714862914892505, error: 0.13937246674615128
sample_space_size(d): 10, kqsci_size(k): 3, gs_energy: -7.8715038046261006, error: 0.139150288

In [17]:
evolved_state_file = "state_probabilities_pi.txt"
state_probabilities = read_states(evolved_state_file)
with open("LiH_KQSCI_result_d_k_pi.txt", "w") as file:
    for j in range(3, max_k):
        for i in range(1, max_d):
            
            sample_space_size = i
            kqsci_size = j
            num_qubits = 12
            
            eigenvalues, eigenvectors = kqsci_eign(state_probabilities, num_qubits, sample_space_size, kqsci_size, qubit_hamiltonian)
            gs_energy = eigenvalues[0] + 0.998447567774
            error = (7.882472287557 + eigenvalues[0] + 0.998447567774) / 7.882472287557 * 100
            file.write(f"sample_space_size(d): {sample_space_size}, kqsci_size(k): {kqsci_size}, gs_energy: {gs_energy}, error: {error}\n")
            print(f"sample_space_size(d): {sample_space_size}, kqsci_size(k): {kqsci_size}, gs_energy: {gs_energy}, error: {error}")
            file.flush()  # Ensure the data is written to the file immediately

sample_space_size(d): 1, kqsci_size(k): 3, gs_energy: -7.862174819467984, error: 0.2575012933576394
sample_space_size(d): 2, kqsci_size(k): 3, gs_energy: -7.876631337584894, error: 0.07410048217139709
sample_space_size(d): 3, kqsci_size(k): 3, gs_energy: -7.880265909442518, error: 0.027990940329281247
sample_space_size(d): 4, kqsci_size(k): 3, gs_energy: -7.8802535253920025, error: 0.028148049039137205
sample_space_size(d): 5, kqsci_size(k): 3, gs_energy: -7.871914466417878, error: 0.1339404790016037
sample_space_size(d): 6, kqsci_size(k): 3, gs_energy: -7.872502033887527, error: 0.12648637769665017
sample_space_size(d): 7, kqsci_size(k): 3, gs_energy: -7.8723139802622315, error: 0.1288720965223629
sample_space_size(d): 8, kqsci_size(k): 3, gs_energy: -7.871976188391483, error: 0.13315745089375455
sample_space_size(d): 9, kqsci_size(k): 3, gs_energy: -7.871843120281574, error: 0.1348456028472834
sample_space_size(d): 10, kqsci_size(k): 3, gs_energy: -7.871637671745369, error: 0.1374520

In [11]:
def selection_full(state_probabilities):
    result = []
    print("Before normalising the states:")
    for time in sorted(state_probabilities.keys()):
        # Append all the states from all times
        for state, probability in state_probabilities[time].items():
            if state not in result:
                result.append(state)
            else:
                continue
    return result

In [12]:
states_chis = selection_full(state_probabilities)
len(states_chis)

Before normalising the states:


42

In [13]:
states= states_chis[:20]

In [14]:
eignva, eignve = kqsci_solver_full(qubit_hamiltonian, states, "E_sub_matrix.txt")

In [15]:
eignva[0] + 0.998447567774

np.float64(-7.88202472818461)

In [16]:
from KQSCI_module import kqsci_solver_full_parallel

In [17]:
states = states_chis[:30]
states

['000011000011',
 '100001100001',
 '000101100001',
 '100001000101',
 '001001001001',
 '010001010001',
 '000101000101',
 '010001010010',
 '010010010010',
 '001010001010',
 '000110000110',
 '010010010001',
 '100010100010',
 '100100100100',
 '001010001001',
 '001001001010',
 '101000101000',
 '000110100010',
 '000101100010',
 '100010100100',
 '100010000011',
 '000110000011',
 '000011100010',
 '100010000110',
 '110000010010',
 '100001000011',
 '010010010100',
 '000101000011',
 '000011000110',
 '000110100001']

In [18]:
eigenvalues, eigenvectors = kqsci_solver_full_parallel(qubit_hamiltonian, states, "E_sub_matrix_parallel.txt", -1)

In [19]:
eigenvalues[0] + 0.998447567774

np.float64(-7.882247963354733)

In [20]:
states = states_chis[:10]
eigenvalues, eigenvectors = kqsci_solver_full_parallel(qubit_hamiltonian, states, "E_sub_matrix_parallel_10.txt", -1)
eigenvalues[0] + 0.998447567774

np.float64(-7.881871125818264)